## Is it a bird?

The basic steps we'll take are:

1. Use DuckDuckGo to search for images of "bird photos"
1. Use DuckDuckGo to search for images of "forest photos"
1. Fine-tune a pretrained neural network to recognise these two groups
1. Try running this model on a picture of a bird and see if it works.

## Step 1: Download images of birds and non-birds

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_results=200):
    # ddg_images: returns Optional[List[dict]]: DuckDuckGo text search results.
    # example return: {'height': 2400, 'image': 'http://etc.usf.edu/clipart/13500/13570/liberty-tree_13570.tif', 'source': 'Bing', 'thumbnail': 'https://tse2.mm.bing.net/th?id=OIP.4t3AojTiUP6TZ-AFaSfCHAHaJ7&pid=Api', 'title': 'Liberty Tree | ClipArt ETC', 'url': 'http://etc.usf.edu/clipart/13500/13570/liberty-tree_13570.htm', 'width': 1790},
    # L: https://fastcore.fast.ai/foundation.html#l
    # L.itemgot: Create new L with item idx of all items
    return L(ddg_images(term, max_results=max_results)).itemgot('image')


In [ ]:
urls = search_images('forest photos', max_results=1)
urls[0]

In [ ]:
# https://fastdownload.fast.ai/core.html#download_url
from fastdownload import download_url
dest = 'forest.jpg'
fpath = download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(fpath)
im.to_thumb(256,256)

Let's grab 200 examples of each of birds and forest photos

In [ ]:
# download_images: https://docs.fast.ai/vision.utils.html

searchQueries = 'forest','bird'
path = Path('bird_or_not')
from time import sleep

for sq in searchQueries:
    dest = (path/sq)
    dest.mkdir(exist_ok=True, parents=True)
    
    urls = search_images(f'{sq} photo')
    download_images(dest, urls=urls)
    sleep(10)  # Pause between searches to avoid over-loading server
    urls = search_images(f'{sq} sun photo')
    download_images(dest, urls=urls)
    sleep(10)  
    urls = search_images(f'{sq} shade photo')
    download_images(dest, urls=urls)
    resize_images(path/sq, max_size=400, dest=path/sq)

## Step 2: Train our model

In [ ]:
# Vision utils: https://docs.fast.ai/vision.utils.html
# L: https://fastcore.fast.ai/foundation.html#l

# Get image files in path recursively, only in folders, if specified.
imfiles = get_image_files(path)

# Find images in fns that can’t be opened
failed = verify_images(imfiles)
# Create new L with f applied to all items. unlink: Remove this file or symbolic link
failed.map(Path.unlink)        
len(failed)

To train a model, we'll need `DataLoaders`, which is an iterator that provides a stream of mini-batches, where each mini-batch is a couple of batches of independent variables and a batch of dependent variables.

To build a DataBlock, there are several steps that needs to be followed. These steps can be asked in the form of questions 
1. What is the types of your input/labels? `Blocks`
2. Where is your data? `get_items`
3. Does something need to be applied to inputs/labels? `get_x, get_y`
4. How to split the data? `splitter`
5. Do we need to apply something on formed items? `item_tfms`
6. Do we need to apply something on formed batches? `batch_tfms`

In [ ]:
# DataBlock: Generic container to quickly build Datasets and DataLoaders.
#            blocks(List): One or more Transform blocks.
#                          blocks are used to define a pre-defined problem domain.
#                          e.g, ImageBlock, CategoryBlock, MultiCategoryBlock, TextBlock etc
#                          CategoryBlock: TransformBlock for single-label categorical targets
#            get_items:    Where is the data?
#                          We can use get_image_files function to go grab all the file locations 
#                          of our images.
#            get_y:        How you extract labels. 
#            splitter:     How to split your data. This is usually a random split between the training and 
#                          validation dataset.
#            item_tfms:    Item transform applied on an individual item basis. This is done on the CPU.

dls = DataBlock(
    # The inputs to our model are images, and the outputs are categories
    blocks=(ImageBlock, CategoryBlock),
    # To find all the inputs to our model, run the get_image_files function (which returns a list of all image files in a path).
    get_items=get_image_files,
    # The labels (y values) is the name of the parent of each file (i.e. the name of the folder they're in, which will be bird or forest).
    get_y=parent_label,
    # Split the data into training and validation sets randomly, using 20% of the data for the validation set.
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    # Before training, resize each image to 192x192 pixels by "squishing" it (as opposed to cropping it).
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)
dls.show_batch(max_n=6)

Now we're ready to train our model. The fastest widely used computer vision model is `resnet18`. 
fastai comes with a helpful `fine_tune()` method which automatically uses best practices for fine tuning pre-trained model, so we'll use that

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(9)

"Fine-tuning" a model means that we're starting with a model someone else has trained using some other dataset (called the *pretrained model*), and adjusting the weights a little bit so that the model learns to recognise your particular dataset. In this case, the pretrained model was trained to recognise photos in *imagenet*, and widely-used computer vision dataset with images covering 1000 categories)

In [ ]:
## Step 3: Use our model (and build your own!)

In [ ]:
is_bird, _, probs = learn.predict('frst.jpg')
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")


